<a href="https://colab.research.google.com/github/snipaid-nlg/models/blob/main/IGEL_001_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing IGEL-001
Evaluating the bloom-6b4-clp-german finetuned on machine-translated german 52k self instruct dataset from Stanford Alpaca.

In [ ]:
!nvidia-smi

Wed Apr  5 17:30:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installs and Imports

In [ ]:
!pip install -q transformers datasets accelerate bitsandbytes sentencepiece
!pip install -q git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Loading the model

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("philschmid/instruct-igel-001")
model = AutoModelForCausalLM.from_pretrained(
    "malteos/bloom-6b4-clp-german",
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "philschmid/instruct-igel-001")

In [ ]:
def generate_prompt(instruction, input=None):
  if input:
    return f'### Anweisung: Erfüllen Sie die Aufgabe am Ende des Textes anhand der folgenden Zusammenhänge im Text. Wenn Sie die Antwort nicht wissen, sagen Sie, dass Sie es nicht wissen, versuchen Sie nicht, eine Antwort zu erfinden.\n"{input}"\n{instruction}\n\n### Antwort:'
  else:
    return f'### Anweisung:\n{instruction}\n\n### Antwort:'

In [ ]:
from transformers import GenerationConfig

generation_config = GenerationConfig(
    top_p=0.9,
    top_k=0,
    temperature=1,
    do_sample=True,
    early_stopping=True,
    length_penalty=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Antwort:", output.split("### Antwort:")[1].strip())

## Artikel per URL einlesen

In [ ]:
!pip install -q newspaper3k

In [ ]:
#@markdown Lade einen Artikel aus dem Internet.

from newspaper import Article

url = 'https://t3n.de/news/hippocamera-app-gegen-demenz-1535827/' #@param {type:"string"}
article = Article(url, browser_user_agent="Googlebot-News")
article.download()
article.parse()
article.text

## Generate

In [ ]:
evaluate("Was ist ein Teaser?")

Antwort: Ein Teaser ist ein kurzer Absatz mit relevanten Informationen. Es kann sowohl im Internet als auch im Fernsehen oder in anderen Medien veröffentlicht werden, um potenzielle Zuschauer oder Zuhörer auf einen längeren Artikel, einen längeren Artikel oder ein Video aufmerksam zu machen. Es ist ein wesentliches Element in vielen Formen der Medienproduktion.<|endoftext|>


In [ ]:
evaluate(input("Anweisung: "), article.text[:400])

Anweisung: Schreibe einen Teaser zu dem Artikel.
Antwort: Dieses Training Ihres Gehirns wird Ihnen im Alter helfen! Denken Sie daran, wie sich Ihr Gehirn an Dinge erinnert, und verwenden Sie diese Taktik, um Ihr Gedächtnis in Schwung zu halten.<|endoftext|>
